# CPE Format Converter (POC) example Jupyter Notebook

This is a proof of concept for a CPE Format Converter.

## Description

Due to time constraints, the construction logic and thought process of this Proof of Concept (PoC) will be initially presented in this Jupyter Notebook. Further development and maintenance will continue to implement more comprehensive functionalities, such as automation and a Web User Interface.

> 由於時間限制，此概念驗證（PoC）的建構邏輯與思路將以本 Jupyter Notebook 的形式初步呈現。後續將持續進行開發與維護，以實現自動化、Web 使用者介面等更完善的功能。

## Design

This Proof of Concept (PoC) aims to provide users with a tool to search for vulnerabilities associated with specific products by inputting Vendor, Product, and Version information. The tool will interact with the National Vulnerability Database (NVD) Common Vulnerabilities and Exposures (CVE) database to retrieve relevant CVE vulnerability data. Since the NVD Vulnerability API (CVE API) uses Common Platform Enumeration (CPE) for searching, user input needs to be converted into CPE format. However, while the CPE format has a standardized specification, there is no standard conversion process, and it cannot be guaranteed that the converted CPE is included in the NVD. To address this issue, we will use the NVD's CPE Directory List as a foundation to convert user input and obtain or generate the correct CPE, subsequently using the NVD Vulnerability API to retrieve further vulnerability data.

> 本概念驗證（PoC）旨在提供使用者一個工具，透過輸入廠商（Vendor）、產品（Product）和版本（Version）資訊，搜尋相關產品的漏洞（Vulnerability）。此工具將與美國國家漏洞資料庫（NVD）的通用漏洞披露（CVE）資料庫互動，取得相關的 CVE 漏洞資訊。由於 NVD 的漏洞應用程式介面（CVE API）使用通用平台列舉（CPE）進行搜尋，因此需要將使用者輸入轉換為 CPE 格式。然而，儘管 CPE 格式具有標準化的規範，卻缺乏標準的轉換流程，且無法確定轉換後的 CPE 是否已被 NVD 收錄。為解決此問題，我們將利用 NVD 提供的 CPE 目錄列表（CPE Directory List）作為基礎，進行使用者輸入的轉換，並取得或產生正確的 CPE，進而使用 NVD 的漏洞應用程式介面取得後續的漏洞資料。

## Usage

We will leverage the interactive features of Jupyter Notebook to provide detailed explanations of the operational process through segmented code, allowing users to reproduce the same operational results by executing this notebook. Please follow the subsequent operational instructions and code comments for adjustments and execution.

> 我們將利用 Jupyter Notebook 的互動式特性，以分段程式碼的方式詳細說明操作流程，並允許使用者透過執行本 Notebook 複現相同的操作結果。請依照後續的操作說明與程式碼註解進行調整與執行。

## Step 1: Get Offical CPE Directory List

Please manually download the CPE Directory List from the official National Vulnerability Database (NVD) website and place the file in the `data` directory of the project repository (Repo). If you have any questions about the operation, please refer to the README file in the `data` directory.

> 請手動自美國國家漏洞資料庫（NVD）官方網站下載 CPE 目錄列表（CPE Directory List），並將檔案放置於專案儲存庫（Repo）的 `data` 目錄中。若對操作方式有疑問，請參閱 `data` 目錄中的 README 檔案。

## Step 2: Decompression and Data Pre-processing

The Common Platform Enumeration (CPE) list provided by the National Vulnerability Database (NVD) is in XML format. While detailed and comprehensive, its structure is not ideal for direct import into Chromadb for subsequent operations. Therefore, we will filter and organize the XML content of the CPE list and output it in Comma-Separated Values (CSV) format.

(Converting to CSV format is a non-essential transitional step, primarily to facilitate the review of the organized XML data and accelerate data utilization and analysis.)

The resulting structured CSV data (or Dataframe) will be processed and imported into Chromadb in the subsequent steps.


> 由於美國國家漏洞資料庫（NVD）提供的通用平台列舉（CPE）列表為 XML 格式，其結構雖詳盡完整，但不利於直接匯入 Chromadb 進行後續操作。因此，我們將對 CPE 列表的 XML 內容進行篩選與整理，並轉換為逗號分隔值（CSV）格式輸出。
> 
> （轉換為 CSV 格式為一非必要的過渡步驟，主要目的是為了方便檢視整理後的 XML 資料，以加速資料的使用與分析。）
> 
> 轉換後的 CSV 結構化資料（或 Dataframe）將於後續步驟中進行計算並匯入 Chromadb。

In [1]:
from lxml import etree
import json
import csv
from typing import Dict, List, Iterator, Optional, Tuple
import logging
from pathlib import Path
import re
from dataclasses import dataclass, asdict
from contextlib import contextmanager


@dataclass
class CPEItem:
    """代表單個 CPE 項目的資料結構"""
    cpe22Uri: str
    cpe23Uri: str
    vendor: str
    product: str
    version: str
    title: str
    references: str


class CPEParser:
    """CPE XML 字典的分析器，用於將 CPE 項目轉換為結構化資料"""

    # CPE 命名空間定義
    NAMESPACES = {
        'cpe-23': 'http://scap.nist.gov/schema/cpe-extension/2.3',
        'def': 'http://cpe.mitre.org/dictionary/2.0',
        'xml': 'http://www.w3.org/XML/1998/namespace'
    }

    def __init__(self, xml_file_path: str):
        """初始化 CPE 分析器

        Args:
            xml_file_path (str): XML 檔案路徑
        """
        self.xml_file_path = Path(xml_file_path)
        self.logger = logging.getLogger(__name__)

        if not self.xml_file_path.exists():
            raise FileNotFoundError(f"找不到 XML 檔案: {xml_file_path}")

    @staticmethod
    def parse_cpe23_uri(cpe23uri: str) -> Dict[str, str]:
        """分析 CPE 2.3 URI 並提取各個組件

        Args:
            cpe23uri (str): CPE 2.3 URI 字串 (例如: cpe:2.3:a:vendor:product:version:...)

        Returns:
            Dict[str, str]: 包含廠商、產品和版本資訊的字典
        """
        # CPE 2.3 格式: cpe:2.3:part:vendor:product:version:update:edition:language:sw_edition:target_sw:target_hw:other
        components = cpe23uri.split(':')

        if len(components) < 6:
            return {"vendor": "", "product": "", "version": ""}

        return {
            'vendor': components[3],
            'product': components[4],
            'version': components[5]
        }

    @contextmanager
    def _open_xml_context(self) -> Iterator:
        """建立用於迭代 XML 檔案中 CPE 項目的上下文管理器

        Yields:
            Iterator: 用於迭代 XML 檔案中 CPE 項目的迭代器
        """
        try:
            context = etree.iterparse(
                str(self.xml_file_path),
                events=('end',),
                tag=f'{{{self.NAMESPACES["def"]}}}cpe-item'
            )
            yield context
        except etree.ParseError as e:
            self.logger.error(f"XML 分析錯誤: {e}")
            raise
        except Exception as e:
            self.logger.error(f"處理 XML 時發生未預期的錯誤: {e}")
            raise

    def _extract_item_data(self, item: etree._Element) -> Optional[CPEItem]:
        """從 XML 元素中提取 CPE 項目資料

        Args:
            item (etree._Element): 包含 CPE 項目資料的 XML 元素

        Returns:
            Optional[CPEItem]: 如果成功提取則返回 CPEItem 物件，否則返回 None
        """
        try:
            # 取得 CPE 2.3 URI
            cpe23_item = item.find('.//cpe-23:cpe23-item', self.NAMESPACES)
            if cpe23_item is None:
                return None

            cpe23uri = cpe23_item.get('name', '')
            if not cpe23uri:
                return None

            # 分析 CPE 組件
            cpe_components = self.parse_cpe23_uri(cpe23uri)

            # 取得標題
            title_elem = item.find(
                './/def:title[@xml:lang="en-US"]', self.NAMESPACES)
            title = title_elem.text if title_elem is not None else ""

            # 取得參考資料
            references = self._extract_references(item)

            # 建立 CPE 項目物件
            return CPEItem(
                cpe22Uri=item.get('name', ''),
                cpe23Uri=cpe23uri,
                vendor=cpe_components['vendor'],
                product=cpe_components['product'],
                version=cpe_components['version'],
                title=title,
                references=references
            )
        except Exception as e:
            self.logger.warning(f"提取 CPE 項目時發生錯誤: {e}")
            return None

    def _extract_references(self, item: etree._Element) -> str:
        """從 XML 元素中提取參考資料

        Args:
            item (etree._Element): 包含參考資料的 XML 元素

        Returns:
            str: 格式化的參考資料字串
        """
        refs = item.findall('.//def:reference', self.NAMESPACES)
        ref_texts = []

        for ref in refs:
            href = ref.get('href', '')
            ref_text = ref.text or ''

            if href and ref_text:
                ref_texts.append(f"{ref_text} ({href})")
            elif href:
                ref_texts.append(href)
            elif ref_text:
                ref_texts.append(ref_text)

        return ' '.join(ref_texts)

    def _clear_element_memory(self, item: etree._Element) -> None:
        """清理 XML 元素佔用的記憶體

        Args:
            item (etree._Element): 要清理的 XML 元素
        """
        item.clear()
        # 清理前面的同層級元素以釋放記憶體
        while item.getprevious() is not None:
            del item.getparent()[0]

    def parse_xml(self) -> List[Dict]:
        """分析整個 CPE XML 檔案

        Returns:
            List[Dict]: 分析後的 CPE 資料列表
        """
        self.logger.info(f"開始分析 XML 檔案: {self.xml_file_path}")
        cpe_items = []

        with self._open_xml_context() as context:
            for event, item in context:
                cpe_item = self._extract_item_data(item)
                if cpe_item:
                    cpe_items.append(asdict(cpe_item))

                self._clear_element_memory(item)

        self.logger.info(f"分析完成，共提取 {len(cpe_items)} 個 CPE 項目")
        return cpe_items

    def process_by_chunk(self, chunk_size: int = 1000) -> Iterator[List[Dict]]:
        """使用分批處理方式處理大型 XML 檔案

        Args:
            chunk_size (int, optional): 每批處理的項目數量. 預設為 1000.

        Yields:
            List[Dict]: 每批處理的 CPE 項目列表
        """
        self.logger.info(f"開始分批處理 XML 檔案 (每批 {chunk_size} 個項目)")
        items = []
        total_processed = 0

        with self._open_xml_context() as context:
            for event, item in context:
                cpe_item = self._extract_item_data(item)
                if cpe_item:
                    items.append(asdict(cpe_item))

                self._clear_element_memory(item)

                if len(items) >= chunk_size:
                    total_processed += len(items)
                    self.logger.debug(f"已處理 {total_processed} 個項目")
                    yield items
                    items = []

            if items:
                total_processed += len(items)
                self.logger.debug(f"已處理 {total_processed} 個項目")
                yield items

        self.logger.info(f"分批處理完成，總共處理 {total_processed} 個項目")

    def save_to_json(self, output_file: str, items: Optional[List[Dict]] = None) -> None:
        """將分析結果儲存為 JSON 格式

        Args:
            output_file (str): 輸出檔案路徑
            items (Optional[List[Dict]], optional): 要儲存的項目列表，如果為 None 則會分析 XML 檔案. 預設為 None.
        """
        output_path = Path(output_file)
        output_path.parent.mkdir(parents=True, exist_ok=True)

        try:
            cpe_items = items if items is not None else self.parse_xml()

            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(cpe_items, f, ensure_ascii=False, indent=2)

            self.logger.info(
                f"已成功儲存 {len(cpe_items)} 個項目至 JSON 檔案: {output_path}")

        except Exception as e:
            self.logger.error(f"儲存 JSON 檔案時發生錯誤: {e}")
            raise

    def save_to_csv(self, output_file: str, items: Optional[List[Dict]] = None) -> None:
        """將分析結果儲存為 CSV 格式

        Args:
            output_file (str): 輸出檔案路徑
            items (Optional[List[Dict]], optional): 要儲存的項目列表，如果為 None 則會分析 XML 檔案. 預設為 None.
        """
        output_path = Path(output_file)
        output_path.parent.mkdir(parents=True, exist_ok=True)

        try:
            cpe_items = items if items is not None else self.parse_xml()
            fieldnames = ['cpe22Uri', 'cpe23Uri', 'vendor', 'product',
                          'version', 'title', 'references']

            with open(output_path, 'w', newline='', encoding='utf-8') as f:
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(cpe_items)

            self.logger.info(
                f"已成功儲存 {len(cpe_items)} 個項目至 CSV 檔案: {output_path}")

        except Exception as e:
            self.logger.error(f"儲存 CSV 檔案時發生錯誤: {e}")
            raise


def setup_logging(log_level=logging.INFO, log_file=None):
    """設定日誌記錄

    Args:
        log_level (int, optional): 日誌等級. 預設為 logging.INFO.
        log_file (str, optional): 日誌檔案路徑，如果為 None 則只會輸出到控制台. 預設為 None.
    """
    log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'

    # 建立日誌處理器
    handlers = []
    handlers.append(logging.StreamHandler())  # 控制台輸出

    if log_file:
        log_dir = Path(log_file).parent
        log_dir.mkdir(parents=True, exist_ok=True)
        handlers.append(logging.FileHandler(
            log_file, encoding='utf-8'))  # 檔案輸出

    # 設定根日誌記錄器
    logging.basicConfig(
        level=log_level,
        format=log_format,
        handlers=handlers
    )


def main():
    # 設定日誌
    log_file = "../logs/cpe_parser.log"
    setup_logging(log_level=logging.INFO, log_file=log_file)
    logger = logging.getLogger(__name__)

    # 設定檔案路徑
    input_file = "../data/official-cpe-dictionary_v2.3.xml"
    output_dir = Path("../data/convert")
    output_dir.mkdir(parents=True, exist_ok=True)

    output_json = output_dir / "official-cpe-dictionary_v2.3_v2.json"
    output_csv = output_dir / "official-cpe-dictionary_v2.3_v2.csv"

    try:
        logger.info(f"開始處理 CPE 字典檔案: {input_file}")
        parser = CPEParser(input_file)

        # 使用分批處理分析 XML 檔案並儲存結果
        all_items = []
        total_processed = 0

        for chunk in parser.process_by_chunk(chunk_size=1000):
            all_items.extend(chunk)
            total_processed += len(chunk)
            logger.info(f"已處理 {total_processed} 個項目")

        # 儲存為 JSON 和 CSV
        parser.save_to_json(str(output_json), all_items)
        parser.save_to_csv(str(output_csv), all_items)

        logger.info("程式執行完成")

    except FileNotFoundError as e:
        logger.error(f"找不到檔案: {e}")
    except Exception as e:
        logger.error(f"程式執行失敗: {e}", exc_info=True)


if __name__ == "__main__":
    main()

2025-03-12 11:23:05,588 - __main__ - INFO - 開始處理 CPE 字典檔案: ../data/official-cpe-dictionary_v2.3.xml
2025-03-12 11:23:05,589 - __main__ - INFO - 開始分批處理 XML 檔案 (每批 1000 個項目)
2025-03-12 11:23:05,607 - __main__ - INFO - 已處理 1000 個項目
2025-03-12 11:23:05,638 - __main__ - INFO - 已處理 2000 個項目
2025-03-12 11:23:05,657 - __main__ - INFO - 已處理 3000 個項目
2025-03-12 11:23:05,674 - __main__ - INFO - 已處理 4000 個項目
2025-03-12 11:23:05,690 - __main__ - INFO - 已處理 5000 個項目
2025-03-12 11:23:05,707 - __main__ - INFO - 已處理 6000 個項目
2025-03-12 11:23:05,724 - __main__ - INFO - 已處理 7000 個項目
2025-03-12 11:23:05,740 - __main__ - INFO - 已處理 8000 個項目
2025-03-12 11:23:05,757 - __main__ - INFO - 已處理 9000 個項目
2025-03-12 11:23:05,774 - __main__ - INFO - 已處理 10000 個項目
2025-03-12 11:23:05,791 - __main__ - INFO - 已處理 11000 個項目
2025-03-12 11:23:05,807 - __main__ - INFO - 已處理 12000 個項目
2025-03-12 11:23:05,825 - __main__ - INFO - 已處理 13000 個項目
2025-03-12 11:23:05,843 - __main__ - INFO - 已處理 14000 個項目
2025-03-12 11:23:05,858 -

### Example of converted format

#### JSON Format

```JSON
{
  "cpe22Uri": "cpe:/a:%240.99_kindle_books_project:%240.99_kindle_books:6::~~~android~~",
  "cpe23Uri": "cpe:2.3:a:\\$0.99_kindle_books_project:\\$0.99_kindle_books:6:*:*:*:*:android:*:*",
  "vendor": "\\$0.99_kindle_books_project",
  "product": "\\$0.99_kindle_books",
  "version": "6",
  "title": "$0.99 Kindle Books project $0.99 Kindle Books (aka com.kindle.books.for99) for android 6.0",
  "references": "Product information (https://play.google.com/store/apps/details?id=com.kindle.books.for99) Government Advisory (https://docs.google.com/spreadsheets/d/1t5GXwjw82SyunALVJb2w0zi3FoLRIkfGPc7AMjRF0r4/edit?pli=1#gid=1053404143)"
},
```

#### CSV Format

```CSV
cpe22Uri,cpe23Uri,vendor,product,version,title,references
cpe:/a:%240.99_kindle_books_project:%240.99_kindle_books:6::~~~android~~,cpe:2.3:a:\$0.99_kindle_books_project:\$0.99_kindle_books:6:*:*:*:*:android:*:*,\$0.99_kindle_books_project,\$0.99_kindle_books,6,$0.99 Kindle Books project $0.99 Kindle Books (aka com.kindle.books.for99) for android 6.0,Product information (https://play.google.com/store/apps/details?id=com.kindle.books.for99) Government Advisory (https://docs.google.com/spreadsheets/d/1t5GXwjw82SyunALVJb2w0zi3FoLRIkfGPc7AMjRF0r4/edit?pli=1#gid=1053404143)
```

## Step 3: Create a vector database (using Chromadb as an example)

To quickly retrieve the closest Common Platform Enumeration (CPE) data based on user input, we will utilize the similarity search capabilities of a vector database. Vector databases excel at transforming complex data (such as images, audio, etc.) into vector representations for mathematical operations. Given the variability of user input, the potential for different registration formats of vendor and product names within the CPE database, and the varying relevance of references included in CPE entries, we aim to leverage the similarity comparison capabilities of the vector database to effectively identify the desired CPE entries. Therefore, we need to import the National Vulnerability Database (NVD) CPE Directory List into Chromadb and convert it into vectors for subsequent similarity comparisons.

> 為了能根據使用者輸入，快速取得最接近的通用平台列舉（CPE）資料，我們將利用向量資料庫的相似性搜尋功能。向量資料庫擅長將複雜資料（如圖像、聲音等）轉換為可進行數學運算的向量表示。由於使用者輸入的多樣性，以及廠商、產品名稱在 CPE 資料庫中可能存在不同形式的登記方式，加上 CPE 條目中包含的參考資料（References）有時相關、有時不相關，我們希望藉由向量資料庫的相似性比較能力，有效識別出期望的 CPE 條目。因此，我們需要將美國國家漏洞資料庫（NVD）的 CPE 目錄列表匯入 Chromadb，並將其轉換為向量，以便後續的相似性比對。

### Structure of the created to vector database

```Markdown
          (Periodic or Manual)
┌───────────────────────────────────────┐
│    NVD CPE Directory (ZIP/XML)        │
└───────────────────────────────────────┘
               │
               ▼
     CPE Directory Updater
               │
               ▼
     CPE Parser & Transformer
               │  (structured JSON)
               ▼
     Embedding Service (Huggingface)
               │ (vector)
               ▼
  ┌────────────────────────────────────┐
  │         Chromadb Vector DB         │
  └────────────────────────────────────┘
```

In [10]:
import json
import logging
import os
import csv
from typing import Tuple, List, Dict, Any, Optional, Union
from pathlib import Path
import chromadb
from chromadb.utils import embedding_functions
from dataclasses import dataclass
import datetime


# 設定日誌
def setup_logging(log_level=logging.INFO, log_file=None):
    """設定日誌記錄

    Args:
        log_level (int, optional): 日誌等級. 預設為 logging.INFO.
        log_file (str, optional): 日誌檔案路徑，如果為 None 則只會輸出到控制台. 預設為 None.
    """
    log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'

    # 建立日誌處理器
    handlers = []
    handlers.append(logging.StreamHandler())  # 控制台輸出

    if log_file:
        log_dir = Path(log_file).parent
        log_dir.mkdir(parents=True, exist_ok=True)
        handlers.append(logging.FileHandler(
            log_file, encoding='utf-8'))  # 檔案輸出

    # 設定根日誌記錄器
    logging.basicConfig(
        level=log_level,
        format=log_format,
        handlers=handlers
    )


# 設定日誌
setup_logging(logging.INFO, "../logs/cpe_database.log")
logger = logging.getLogger(__name__)


@dataclass
class CPEItem:
    """代表單個 CPE 項目的資料結構"""
    cpe22Uri: str
    cpe23Uri: str
    vendor: str
    product: str
    version: str
    title: str
    references: str


class EmbeddingModelProvider:
    """提供不同的嵌入模型選項"""

    @staticmethod
    def get_default_embedding():
        """獲取預設的嵌入函數

        Returns:
            embedding_function: 預設的嵌入函數
        """
        return embedding_functions.DefaultEmbeddingFunction()

    # TODO: 此區域程式碼修正中，後續引入 GPU 加速時完成修正
    # @staticmethod
    # def get_huggingface_embedding(model_name: str = "sentence-transformers/all-MiniLM-L6-v2", use_gpu: bool = False):
    #     """獲取基於 HuggingFace 的嵌入函數

    #     Args:
    #         model_name (str, optional): 模型名稱. 預設為 "sentence-transformers/all-MiniLM-L6-v2".
    #         use_gpu (bool, optional): 是否使用 GPU. 預設為 False.

    #     Returns:
    #         embedding_function: HuggingFace 嵌入函數
    #     """
    #     try:
    #         from sentence_transformers import SentenceTransformer

    #         # 設定設備
    #         device = "cuda" if use_gpu else "cpu"
    #         logger.info(f"使用 HuggingFace 模型: {model_name}，設備: {device}")

    #         model = SentenceTransformer(model_name, device=device)

    #         # 建立自訂的嵌入函數
    #         def huggingface_embedder(texts):
    #             return model.encode(texts).tolist()

    #         return embedding_functions.EmbeddingFunction(huggingface_embedder)

    #     except ImportError:
    #         logger.warning("未安裝 sentence-transformers 庫，請使用 pip install sentence-transformers 進行安裝")
    #         logger.warning("將使用預設嵌入函數代替")
    #         return EmbeddingModelProvider.get_default_embedding()
    #     except Exception as e:
    #         logger.error(f"加載 HuggingFace 模型時發生錯誤: {str(e)}")
    #         logger.warning("將使用預設嵌入函數代替")
    #         return EmbeddingModelProvider.get_default_embedding()


class CPEDatabaseManager:
    def __init__(self,
                 db_path: str = "../database/cpe_database",
                 use_gpu: bool = False,
                 embedding_model: str = "default"):
        """初始化 CPE 資料庫管理器

        Args:
            db_path (str): 資料庫存放路徑
            use_gpu (bool, optional): 是否使用 GPU. 預設為 False.
            embedding_model (str, optional): 要使用的嵌入模型類型.
                                             可選值: "default", "huggingface". 預設為 "default".
        """
        self.db_path = Path(db_path)
        self.db_path.mkdir(parents=True, exist_ok=True)

        self.client = None
        self.collection = None

        # 設定 GPU 使用選項
        self.use_gpu = use_gpu

        # 選擇嵌入模型
        self.setup_embedding_function(embedding_model)

        # 設定資料庫
        self.setup_database()

    def setup_embedding_function(self, embedding_model: str):
        """設定嵌入函數

        Args:
            embedding_model (str): 嵌入模型類型
        """
        # if embedding_model.lower() == "huggingface":
        #     self.embedding_function = EmbeddingModelProvider.get_huggingface_embedding(use_gpu=self.use_gpu)
        #     device_info = "GPU" if self.use_gpu else "CPU"
        #     embedding_description = f"HuggingFace (device: {device_info})"
        # else:
        #     self.embedding_function = EmbeddingModelProvider.get_default_embedding()
        #     embedding_description = "預設"
        self.embedding_function = EmbeddingModelProvider.get_default_embedding()
        embedding_description = "預設"

        logger.info(f"使用嵌入模型: {embedding_description}")

    def setup_database(self) -> None:
        """設定資料庫連接和集合"""
        try:
            self.client = chromadb.PersistentClient(path=str(self.db_path))

            # 使用設定的 embedding 函數建立集合
            self.collection = self.client.get_or_create_collection(
                name="cpe_dictionary",
                metadata={"description": "CPE Dictionary Collection"},
                embedding_function=self.embedding_function
            )
            logger.info(f"成功設定資料庫連接，儲存位置: {self.db_path}")
        except Exception as e:
            logger.error(f"設定資料庫時發生錯誤: {str(e)}")
            raise

    def load_cpe_data(self, file_path: Union[str, Path]) -> List[Dict[str, Any]]:
        """讀取並解析 CPE JSON 檔案

        Args:
            file_path (Union[str, Path]): JSON 檔案路徑

        Returns:
            List[Dict[str, Any]]: 解析後的 CPE 資料
        """
        file_path = Path(file_path)
        if not file_path.exists():
            raise FileNotFoundError(f"找不到檔案: {file_path}")

        try:
            logger.info(f"開始讀取 CPE 資料: {file_path}")
            with open(file_path, "r", encoding="utf-8") as file:
                data = json.load(file)
            logger.info(f"成功讀取 {len(data)} 筆 CPE 資料")
            return data
        except json.JSONDecodeError as e:
            logger.error(f"解析 JSON 檔案時發生錯誤: {str(e)}")
            raise
        except Exception as e:
            logger.error(f"讀取 CPE 資料時發生錯誤: {str(e)}")
            raise

    def prepare_search_text(self, item: Dict[str, Any]) -> str:
        """準備用於向量搜尋的文字內容

        這個文字將被轉換為向量，並用於相似度搜尋。
        根據使用者需求，我們優化文字內容以支援廠商、產品和版本的搜尋。

        Args:
            item (Dict[str, Any]): CPE 項目資料

        Returns:
            str: 用於向量搜尋的文字
        """
        # 將最重要的資訊放在前面，並重複以增加權重
        title = item.get('title', '').strip()
        vendor = item.get('vendor', '').strip()
        product = item.get('product', '').strip()
        version = item.get('version', '').strip()

        # 構建優化的搜尋文字
        search_text = f"{vendor} {product} {version} "
        search_text += f"{title} "
        search_text += f"{vendor} {product} "

        # 加入完整的資訊
        search_text += f"""
        Title: {title}
        Vendor: {vendor}
        Product: {product}
        Version: {version}
        """

        return search_text.strip()

    def prepare_documents(self, cpe_data: List[Dict[str, Any]]) -> Tuple[List[str], List[Dict[str, str]], List[str]]:
        """準備要導入到資料庫的文件資料

        Args:
            cpe_data (List[Dict[str, Any]]): CPE 資料列表

        Returns:
            Tuple[List[str], List[Dict[str, str]], List[str]]: 
            返回文件內容、中繼資料和ID的元組
        """
        documents = []
        metadatas = []
        ids = []

        for i, item in enumerate(cpe_data):
            try:
                # 取得 CPE 2.3 URI 作為唯一識別符
                cpe23uri = item.get('cpe23Uri', '')
                if not cpe23uri:
                    logger.warning(f"第 {i} 筆資料缺少 cpe23Uri，將略過")
                    continue

                # 使用 CPE 2.3 URI 作為 ID
                doc_id = cpe23uri

                # 製作優化的文件內容
                doc_content = self.prepare_search_text(item)

                # 準備豐富的中繼資料，方便後續檢索和過濾
                metadata = {
                    'vendor': item.get('vendor', ''),
                    'product': item.get('product', ''),
                    'version': item.get('version', ''),
                    'cpe22Uri': item.get('cpe22Uri', ''),
                    'cpe23Uri': cpe23uri,
                    'title': item.get('title', ''),
                    # 將 references 截斷以避免過長
                    'references': item.get('references', '')[:1000] if item.get('references') else ''
                }

                documents.append(doc_content)
                metadatas.append(metadata)
                ids.append(doc_id)

            except Exception as e:
                logger.warning(f"處理第 {i} 筆資料時發生錯誤: {str(e)}")
                continue

        return documents, metadatas, ids

    def import_data(self, file_path: Union[str, Path], batch_size: int = 1000,
                    reset_collection: bool = False) -> None:
        """導入 CPE 資料到資料庫

        Args:
            file_path (Union[str, Path]): CPE JSON 檔案路徑
            batch_size (int, optional): 批次處理大小. 預設為 1000.
            reset_collection (bool, optional): 是否重置集合. 預設為 False.
        """
        try:
            # 如果需要重置集合
            if reset_collection:
                logger.info("重置集合...")
                # ChromaDB v0.6.0 兼容性處理
                try:
                    collection_names = [
                        col for col in self.client.list_collections()]
                    if "cpe_dictionary" in collection_names:
                        self.client.delete_collection("cpe_dictionary")
                except Exception as e:
                    logger.warning(f"檢查集合時發生錯誤 (可能是 ChromaDB 版本不同): {str(e)}")
                    # 嘗試不同的方法
                    try:
                        self.client.get_collection("cpe_dictionary")
                        self.client.delete_collection("cpe_dictionary")
                    except Exception:
                        # 如果集合不存在，忽略錯誤
                        pass

                self.collection = self.client.create_collection(
                    name="cpe_dictionary",
                    metadata={"description": "CPE Dictionary Collection"},
                    embedding_function=self.embedding_function
                )
                logger.info("集合已重置")

            # 讀取資料
            cpe_data = self.load_cpe_data(file_path)

            # 準備文件
            logger.info("準備文件資料...")
            documents, metadatas, ids = self.prepare_documents(cpe_data)
            logger.info(f"準備了 {len(documents)} 筆有效的 CPE 資料")

            # 批次處理
            total_items = len(documents)
            for i in range(0, total_items, batch_size):
                batch_end = min(i + batch_size, total_items)
                batch_docs = documents[i:batch_end]
                batch_meta = metadatas[i:batch_end]
                batch_ids = ids[i:batch_end]

                logger.info(f"正在導入批次 {i//batch_size + 1}/{(total_items-1)//batch_size + 1} " +
                            f"({i+1}-{batch_end}/{total_items})...")

                try:
                    # 使用 upsert 而非 add，以便處理重複的 CPE ID
                    self.collection.upsert(
                        documents=batch_docs,
                        metadatas=batch_meta,
                        ids=batch_ids
                    )
                    logger.info(f"已處理 {batch_end}/{total_items} 筆資料")
                except Exception as e:
                    logger.error(f"批次處理時發生錯誤: {str(e)}")
                    logger.error(f"問題批次範圍: {i+1}-{batch_end}")
                    # 繼續處理下一批，而不是中斷整個過程
                    continue

            logger.info(f"成功導入 CPE 資料，總計 {total_items} 筆")

        except Exception as e:
            logger.error(f"導入資料時發生錯誤: {str(e)}")
            raise

    def search(self,
               vendor: str = None,
               product: str = None,
               version: str = None,
               n_results: int = 5) -> Dict[str, Any]:
        """搜尋 CPE 資料，僅使用廠商、產品、版本進行搜尋

        Args:
            vendor (str, optional): 廠商名稱，可為空
            product (str, optional): 產品名稱，可為空
            version (str, optional): 版本號，可為空
            n_results (int): 返回結果數量，預設為5

        Returns:
            Dict[str, Any]: 搜尋結果，包含過濾前和過濾後的結果
        """
        try:
            # 確保至少提供一個搜尋條件
            if not any([vendor, product, version]):
                raise ValueError(
                    "必須提供至少一個搜尋條件 (vendor, product, version)")

            # 構建搜尋關鍵字
            search_parts = []
            if vendor:
                search_parts.append(f"{vendor}")
            if product:
                search_parts.append(f"{product}")
            if version:
                search_parts.append(f"{version}")

            search_query = " ".join(search_parts)

            # 使用文本搜尋
            original_results = self.collection.query(
                query_texts=[search_query],
                n_results=n_results,
                include=['metadatas', 'distances']
            )

            # 進行精確過濾
            filtered_indices = []
            for i, metadata in enumerate(original_results['metadatas'][0]):
                match = True
                if vendor and vendor.lower() not in metadata['vendor'].lower():
                    match = False
                if product and product.lower() not in metadata['product'].lower():
                    match = False
                if version and version.lower() not in metadata['version'].lower():
                    match = False

                if match:
                    filtered_indices.append(i)

            # 創建結果字典，包含過濾前後的結果
            results = {
                'original_results': original_results,
                'filtered_results': None,
                'is_filtered': True
            }

            # 根據過濾後的索引重建結果
            if filtered_indices:
                filtered_results = {
                    'ids': [[original_results['ids'][0][i] for i in filtered_indices]],
                    'metadatas': [[original_results['metadatas'][0][i] for i in filtered_indices]],
                    'distances': [[original_results['distances'][0][i] for i in filtered_indices]]
                }

                # 如果原結果包含 documents，也過濾它
                if 'documents' in original_results:
                    filtered_results['documents'] = [
                        [original_results['documents'][0][i] for i in filtered_indices]]

                results['filtered_results'] = filtered_results
            else:
                # 如果過濾後沒有結果，設定空結果
                results['filtered_results'] = {
                    'ids': [[]],
                    'metadatas': [[]],
                    'distances': [[]]
                }

            return results

        except Exception as e:
            logger.error(f"搜尋時發生錯誤: {str(e)}")
            raise

    def get_collection_info(self) -> Dict[str, Any]:
        """獲取集合資訊

        Returns:
            Dict[str, Any]: 集合資訊
        """
        try:
            # 獲取集合計數
            count = self.collection.count()

            # 判斷使用的嵌入模型類型
            try:
                # 嘗試檢查嵌入函數的類型
                if hasattr(embedding_functions, 'DefaultEmbeddingFunction') and \
                   isinstance(self.embedding_function, embedding_functions.DefaultEmbeddingFunction):
                    model_type = "Default"
                else:
                    # 嘗試使用不同的方式判斷
                    model_name = getattr(
                        self.embedding_function, '_model_name', None)
                    if model_name and 'huggingface' in str(model_name).lower():
                        model_type = "HuggingFace"
                    else:
                        model_type = "Default" if not self.use_gpu else "HuggingFace"
            except Exception as e:
                logger.warning(f"判斷嵌入模型類型時發生錯誤: {str(e)}")
                # 回退到基於 use_gpu 的簡單判斷
                model_type = "Default" if not self.use_gpu else "HuggingFace"

            # 獲取集合中繼資料
            collection_info = {
                "name": self.collection.name,
                "metadata": self.collection.metadata,
                "count": count,
                "embedding_model": model_type,
                "device": "GPU" if self.use_gpu else "CPU"
            }

            return collection_info
        except Exception as e:
            logger.error(f"獲取集合資訊時發生錯誤: {str(e)}")
            raise


def format_search_results(results: Dict[str, Any]) -> str:
    """格式化搜尋結果為可讀的文字，包含過濾前和過濾後的結果

    Args:
        results (Dict[str, Any]): 搜尋結果

    Returns:
        str: 格式化的搜尋結果文字
    """
    if not results:
        return "未找到符合的結果"

    output = []

    # 處理原始結果
    original_results = results.get('original_results', {})
    if original_results and original_results.get('metadatas') and original_results['metadatas'][0]:
        output.append("\n=== 原始搜尋結果 ===")
        for i, (metadata, distance) in enumerate(zip(original_results['metadatas'][0],
                                                     original_results.get('distances', [[0] * len(original_results['metadatas'][0])])[0])):
            score = 1.0 - (distance / 2.0)  # 將距離轉換為可理解的分數

            output.append(f"\n原始結果 {i+1} (相似度: {score:.2f}):")
            output.append(f"產品: {metadata['product']}")
            output.append(f"廠商: {metadata['vendor']}")
            output.append(f"版本: {metadata['version']}")
            output.append(f"CPE23: {metadata['cpe23Uri']}")

    # 處理過濾結果
    filtered_results = results.get('filtered_results', {})
    if results.get('is_filtered') and filtered_results and filtered_results.get('metadatas') and filtered_results['metadatas'][0]:
        output.append("\n=== 過濾後的結果 ===")
        for i, (metadata, distance) in enumerate(zip(filtered_results['metadatas'][0],
                                                     filtered_results.get('distances', [[0] * len(filtered_results['metadatas'][0])])[0])):
            score = 1.0 - (distance / 2.0)

            output.append(f"\n過濾結果 {i+1} (相似度: {score:.2f}):")
            output.append(f"產品: {metadata['product']}")
            output.append(f"廠商: {metadata['vendor']}")
            output.append(f"版本: {metadata['version']}")
            output.append(f"CPE23: {metadata['cpe23Uri']}")
    elif results.get('is_filtered') and (not filtered_results or not filtered_results.get('metadatas') or not filtered_results['metadatas'][0]):
        output.append("\n=== 過濾後的結果 ===")
        output.append("過濾後沒有符合條件的結果")

    return "\n".join(output)


def batch_search_from_csv(db_manager, csv_file_path, output_file_path=None):
    """從CSV檔案批次搜尋CPE資料並輸出結果

    Args:
        db_manager: CPE資料庫管理器實例
        csv_file_path (str): 輸入CSV檔案路徑
        output_file_path (str, optional): 輸出檔案路徑。如果為None，會自動生成檔名

    Returns:
        str: 輸出檔案路徑
    """
    if not output_file_path:
        # 生成輸出檔名，包含時間戳記
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file_path = f"search_results_{timestamp}.txt"

    # 統計搜尋資訊
    total_items = 0
    items_with_results = 0

    logger.info(f"開始從 {csv_file_path} 批次搜尋，結果將輸出到 {output_file_path}")

    try:
        with open(csv_file_path, 'r', encoding='utf-8') as csv_file, \
                open(output_file_path, 'w', encoding='utf-8') as output_file:

            # 寫入檔案標頭
            output_file.write(f"CPE資料庫搜尋結果 - 批次處理\n")
            output_file.write(f"輸入檔案: {csv_file_path}\n")
            output_file.write(
                f"處理時間: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            output_file.write(f"{'='*60}\n\n")

            # 讀取CSV
            csv_reader = csv.DictReader(csv_file)

            # 檢查CSV是否有必要的欄位
            required_fields = ['vendor', 'product', 'version']
            missing_fields = [
                field for field in required_fields if field not in csv_reader.fieldnames]
            if missing_fields:
                raise ValueError(f"CSV檔案缺少必要欄位: {', '.join(missing_fields)}")

            # 處理每一行
            for i, row in enumerate(csv_reader, 1):
                total_items += 1

                vendor = row.get('vendor', '').strip() or None
                product = row.get('product', '').strip() or None
                version = row.get('version', '').strip() or None

                # 如果所有欄位都是空的，跳過這一行
                if not any([vendor, product, version]):
                    logger.warning(f"第 {i} 行: 所有搜尋條件為空，跳過")
                    output_file.write(f"項目 {i}: 所有搜尋條件為空，跳過\n\n")
                    continue

                # 寫入當前搜尋項目資訊
                output_file.write(f"項目 {i}:\n")
                output_file.write(f"  廠商: {vendor or '無'}\n")
                output_file.write(f"  產品: {product or '無'}\n")
                output_file.write(f"  版本: {version or '無'}\n\n")

                # 搜尋
                try:
                    results = db_manager.search(
                        vendor=vendor, product=product, version=version)
                    result_text = format_search_results(results)

                    # 檢查是否有搜尋結果
                    has_results = False
                    filtered_results = results.get('filtered_results', {})
                    if filtered_results and filtered_results.get('metadatas') and filtered_results['metadatas'][0]:
                        has_results = True
                        items_with_results += 1

                    # 寫入搜尋結果
                    output_file.write(result_text)
                    output_file.write(f"\n{'='*60}\n\n")

                    # 記錄日誌
                    if has_results:
                        logger.info(
                            f"第 {i} 行: 搜尋成功，找到 {len(filtered_results['metadatas'][0])} 筆結果")
                    else:
                        logger.info(f"第 {i} 行: 搜尋成功，但沒有符合的結果")

                except Exception as e:
                    logger.error(f"第 {i} 行: 搜尋時發生錯誤: {str(e)}")
                    output_file.write(f"搜尋時發生錯誤: {str(e)}\n\n")
                    output_file.write(f"{'='*60}\n\n")

            # 寫入摘要
            output_file.write(f"批次搜尋摘要:\n")
            output_file.write(f"總項目數: {total_items}\n")
            output_file.write(f"有結果的項目數: {items_with_results}\n")
            output_file.write(f"無結果的項目數: {total_items - items_with_results}\n")

        logger.info(f"批次搜尋完成，共處理 {total_items} 項，{items_with_results} 項有結果")
        return output_file_path

    except Exception as e:
        logger.error(f"批次搜尋過程中發生錯誤: {str(e)}")
        raise


def main():
    # 設定檔案路徑
    file_path = "../data/convert/official-cpe-dictionary_v2.3_v2.json"

    try:
        use_gpu = False
        embedding_model = "default"
        logger.info("使用預設嵌入模型 (CPU)")

        # 初始化資料庫管理器
        db_manager = CPEDatabaseManager(
            use_gpu=use_gpu, embedding_model=embedding_model)

        # 匯入資料 (如果需要)
        import_data = input("是否要導入 CPE 資料？(y/n): ").strip().lower() == 'y'
        if import_data:
            reset = input("是否要重置現有集合？(y/n): ").strip().lower() == 'y'
            db_manager.import_data(file_path, reset_collection=reset)

        # 顯示更新後的集合資訊
        collection_info = db_manager.get_collection_info()
        print(f"\n更新後集合資訊:")
        print(f"名稱: {collection_info['name']}")
        print(f"項目數量: {collection_info['count']}")
        print(f"嵌入模型: {collection_info['embedding_model']}")
        print(f"使用裝置: {collection_info['device']}")

        # 選擇搜尋模式
        print("\n請選擇搜尋模式:")
        print("1. 單次搜尋")
        print("2. 從CSV檔案批次搜尋")

        mode = input("請選擇 (1-2): ").strip()

        if mode == "2":
            # 批次搜尋模式
            csv_file_path = input("請輸入CSV檔案路徑: ").strip()
            if not os.path.exists(csv_file_path):
                print(f"找不到檔案: {csv_file_path}")
                return

            output_path = batch_search_from_csv(db_manager, csv_file_path)
            print(f"\n批次搜尋完成! 結果已輸出至: {output_path}")

        else:
            # 單次搜尋模式 (預設)
            while True:
                print("\n請輸入搜尋條件 (直接按 Enter 跳過):")
                vendor = input("廠商名稱: ").strip() or None
                product = input("產品名稱: ").strip() or None
                version = input("版本號: ").strip() or None

                if not any([vendor, product, version]):
                    print("至少需要提供一個搜尋條件")
                    continue

                try:
                    results = db_manager.search(
                        vendor=vendor, product=product, version=version)
                    print(format_search_results(results))

                    if input("\n是否繼續搜尋？(y/n): ").strip().lower() != 'y':
                        break
                except Exception as e:
                    print(f"搜尋過程中發生錯誤: {str(e)}")
                    if input("\n是否繼續搜尋？(y/n): ").strip().lower() != 'y':
                        break

    except Exception as e:
        logger.error(f"程式執行時發生錯誤: {str(e)}")
        raise


if __name__ == "__main__":
    main()

2025-03-13 02:21:20,532 - __main__ - INFO - 使用預設嵌入模型 (CPU)
2025-03-13 02:21:20,534 - __main__ - INFO - 使用嵌入模型: 預設
2025-03-13 02:21:20,541 - __main__ - INFO - 成功設定資料庫連接，儲存位置: ../database/cpe_database
2025-03-13 02:21:24,076 - __main__ - WARNING - 判斷嵌入模型類型時發生錯誤: isinstance() arg 2 must be a type, a tuple of types, or a union



更新後集合資訊:
名稱: cpe_dictionary
項目數量: 1377010
嵌入模型: Default
使用裝置: CPU

請選擇搜尋模式:
1. 單次搜尋
2. 從CSV檔案批次搜尋


2025-03-13 02:21:34,848 - __main__ - INFO - 開始從 ./transform_data.csv 批次搜尋，結果將輸出到 search_results_20250313_022134.txt
2025-03-13 02:21:35,051 - __main__ - INFO - 第 1 行: 搜尋成功，但沒有符合的結果
2025-03-13 02:21:35,132 - __main__ - INFO - 第 2 行: 搜尋成功，但沒有符合的結果
2025-03-13 02:21:35,198 - __main__ - INFO - 第 3 行: 搜尋成功，但沒有符合的結果
2025-03-13 02:21:35,276 - __main__ - INFO - 第 4 行: 搜尋成功，但沒有符合的結果
2025-03-13 02:21:35,332 - __main__ - INFO - 第 5 行: 搜尋成功，但沒有符合的結果
2025-03-13 02:21:35,384 - __main__ - INFO - 第 6 行: 搜尋成功，但沒有符合的結果
2025-03-13 02:21:35,441 - __main__ - INFO - 第 7 行: 搜尋成功，但沒有符合的結果
2025-03-13 02:21:35,513 - __main__ - INFO - 第 8 行: 搜尋成功，但沒有符合的結果
2025-03-13 02:21:35,584 - __main__ - INFO - 第 9 行: 搜尋成功，但沒有符合的結果
2025-03-13 02:21:35,652 - __main__ - INFO - 第 10 行: 搜尋成功，但沒有符合的結果
2025-03-13 02:21:35,707 - __main__ - INFO - 第 11 行: 搜尋成功，但沒有符合的結果
2025-03-13 02:21:35,770 - __main__ - INFO - 第 12 行: 搜尋成功，但沒有符合的結果
2025-03-13 02:21:35,839 - __main__ - INFO - 第 13 行: 搜尋成功，但沒有符合的結果
2025-03-13 02:21:35,896 - __main__ 


批次搜尋完成! 結果已輸出至: search_results_20250313_022134.txt
